# Initial setup

In [1]:
import findspark

findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col,isnan, when, count
import pyspark.sql.functions as F

from configparser import ConfigParser

config = ConfigParser()
# create your own config.ini in root of project folder to store project configurations
config.read('config.ini')

pathfile = config.get('main', 'dirty_csv')  

spark = SparkSession.builder \
    .config("spark.driver.memory", "15g") \
    .appName("SparkFlight").getOrCreate()



Data is the pre-processed data from Twan and Daan

In [2]:
# data = spark.read.csv(pathfile, inferSchema='true', header='true', mode='PERMISSIVE', encoding='ISO-8859-1').limit(10**6).cache()
data = spark.read.csv(pathfile, inferSchema='true', header='true', mode='PERMISSIVE', encoding='ISO-8859-1')

### Analysis cells

In [3]:
data.show()

+-----------------+-------+--------+--------------+--------+----+--------+---------------------------+------------------+------+-------------+-------+------+-------+-------------+-------------------+-------------------+-------------------+-------------------+
|ActualElapsedTime|AirTime|ArrDelay|CRSElapsedTime|DepDelay|Dest|Distance|LateAircraftAndCarrierDelay|NASAndWeatherDelay|Origin|SecurityDelay|TailNum|TaxiIn|TaxiOut|UniqueCarrier|    CRSDepTimeStamp|       DepTimestamp|       ArrTimestamp|    CRSArrTimestamp|
+-----------------+-------+--------+--------------+--------+----+--------+---------------------------+------------------+------+-------------+-------+------+-------+-------------+-------------------+-------------------+-------------------+-------------------+
|               53|     32|      -8|            65|       4| PIT|     205|                          0|                 0|   DCA|            0| N443US|     7|     14|           US|2002-10-10 15:45:00|2002-10-10 15:49:00|2

## Correlation filter
This is my proposed solution for batching. The idea is simple:
1) First query the 16.000 elements
2) Then check if two columns are correlated. And exstend from there 
3) ...
4) Profit

In [4]:
from itertools import permutations

def unique_permutations(iterable, r=None):
    previous = tuple()
    for p in permutations(sorted(iterable), r):
        if p > previous:
            previous = p
            yield p

Step 1: Transform all the categorical columns to numbers

In [5]:
import pandas as pd
import numpy as np

data_limit = data.limit(16000) #this could be a pre-processing step
indexer =  data_limit.select("*").toPandas()
transformed_columns = []
# first all the string columns are mapped to integers
for col in indexer.select_dtypes(exclude=['number', 'datetime']).columns:
     indexer[col] = pd.Categorical(indexer[col], categories=indexer[col].unique()).codes
     transformed_columns.append(col)

# secondly, datetime columns are transformed to seconds
for col in indexer.select_dtypes(include=['datetime']).columns:
    indexer[col] = indexer[col].values.view('<i8')/10**9

# secondly, datetime columns are transformed to seconds
for col in indexer.columns:
    indexer[col] = indexer[col] - indexer[col].min()
    indexer[col] = indexer[col].astype('float')


This is the filter used by Splunk. It uses a map to set the p-value, but it also uses a check if a dependecy is already a soft dependency.

In [6]:
import math
def map_find_sfd_and_pvalue(sampledataset, sfd_threshold = 0.99, sfd_size = 0.95):
    
    def simple_hash(x, max_value, buckets): 
        return np.floor((x / max_value ) * buckets)
    
    # returns a tuple (dependency, possibleSFD, p-value)
    def _map(dependency):
        import pandas as pd
        lhs, rhs = dependency
        
        # this if statement makes it possible to do some optimalisations
        if len(lhs) > 1:
            # first we construct a column for the left hand side. Combine the individual values
            sampledataset["lhs"] = ""
            for item in lhs:
                sampledataset["lhs"] = sampledataset['lhs'] + "|" + sampledataset[item].apply(str)
            
            # now transform those strings into a number and "normalize" it
            sampledataset['lhs'] = pd.Categorical(indexer['lhs'], categories=indexer['lhs'].unique()).codes
            sampledataset["lhs"] = sampledataset["lhs"] - sampledataset["lhs"].min()
        else:
            sampledataset['lhs'] = sampledataset[lhs[0]]
        
        
        # now check if there exists a soft function dependency. We use a threshold value (sfd_threshold) and a 
        # size value (sfd_size). The size value is to filter out any soft functional dependecies that look
        # like they are, but are not. See the CORDS paper for more information. 
        new_frame = sampledataset[['lhs',rhs]]
        new_frame = new_frame.drop_duplicates()
        
        out = []
        # number of unique values is the same as the dataset size. Possible Hard dependency otherwise Soft
        if new_frame['lhs'].nunique() >= sfd_threshold * new_frame['lhs'].size \
            and new_frame.shape[0] <= sfd_size * sampledataset.shape[0]:
            
            out.append((dependency, dependency, new_frame['lhs'].nunique() / new_frame['lhs'].size))
        
        # number of unique values is the same as the dataset size. Possible Hard dependency otherwise Soft
        if len(lhs) == 1 and new_frame[rhs].nunique() >= sfd_threshold * new_frame[rhs].size\
            and new_frame.shape[0] <= sfd_size * sampledataset.shape[0]:
            
            out.append( (dependency, ([rhs], lhs[0]), new_frame[rhs].nunique() / new_frame[rhs].size ))

        if len(out) > 0:
            return out

        # the is no soft functional dependency found, therefore calculate the correlation        
        buckets = math.sqrt(new_frame.shape[0] / 5)
        sampledataset["lhs"] = simple_hash(sampledataset["lhs"], sampledataset["lhs"].max(), buckets)
        sampledataset[rhs] = simple_hash(sampledataset[rhs], sampledataset[rhs].max(), buckets)
        
        crosstab =  pd.crosstab(sampledataset['lhs'], sampledataset[rhs])
        _, p, _, _ = stats.chi2_contingency(crosstab)
        
        return [(dependency, None, p)]
    return _map


def filter_based_on_pvalue(sampledataset):
    def _filter(datasettuple):
        return datasettuple[2] < 0.05 or not datasettuple[1] is None
    return _filter

def find_correlected_dependencies_with_spark(dependencies_to_check):
    rdd=spark.sparkContext.parallelize(dependencies_to_check)
    mapped = rdd.flatMap(map_find_sfd_and_pvalue(indexer))
    filtered = mapped.filter(filter_based_on_pvalue(indexer))
    return filtered.collect()
    

In [7]:
import hashlib
from scipy import stats

# in this list we add all the elements that must be checked by spark
q = []
q2 = []
q3 = []

# We have already seen these combinations
cachedCombinations = []

foundcorrelations = []

# first we add all the single columns. For instance A->B, B->C BUT NOT B-> A
for p in unique_permutations(data.columns, 2):
    if (tuple([p[-1]]),p[0:-1]) not in cachedCombinations and len(p[0:-1]) > 0:
        cachedCombinations.append((p[0:-1],tuple([p[-1]])))
        q.append((list(p[0:-1]),p[-1]))
        
for p in unique_permutations(data.columns, 3):
        q2.append((list(p[0:-1]),p[-1]))
        
for p in unique_permutations(data.columns, 4):
        q3.append((list(p[0:-1]),p[-1]))


Find single dependencies and/or correlations

In [8]:

single_dependencies = find_correlected_dependencies_with_spark(q)  

print(single_dependencies)


[((['ActualElapsedTime'], 'AirTime'), None, 0.0), ((['ActualElapsedTime'], 'ArrDelay'), None, 0.0), ((['ActualElapsedTime'], 'ArrTimestamp'), None, 0.022695925115031806), ((['ActualElapsedTime'], 'CRSArrTimestamp'), None, 0.021728708901639304), ((['ActualElapsedTime'], 'CRSDepTimeStamp'), None, 0.021788529194727662), ((['ActualElapsedTime'], 'CRSElapsedTime'), None, 0.0), ((['ActualElapsedTime'], 'DepTimestamp'), None, 0.020448657957126082), ((['ActualElapsedTime'], 'Dest'), None, 3.268630628663676e-248), ((['ActualElapsedTime'], 'Distance'), None, 0.0), ((['ActualElapsedTime'], 'NASAndWeatherDelay'), None, 0.0), ((['ActualElapsedTime'], 'Origin'), None, 3.146223750490332e-247), ((['ActualElapsedTime'], 'SecurityDelay'), None, 0.005115583593601542), ((['ActualElapsedTime'], 'TailNum'), None, 0.0001083728788888629), ((['ActualElapsedTime'], 'TaxiIn'), None, 0.0003623341912963334), ((['ActualElapsedTime'], 'TaxiOut'), None, 0.0), ((['ActualElapsedTime'], 'UniqueCarrier'), None, 0.0), (([

In [9]:
# THIS IS SOME CODE TO DISPLAY THE RESULTS. NO INTERESTING LOGIC IS IN HERE
from pandas.api.types import is_numeric_dtype

single_softFD = list(map(lambda x: f"{x[1][0][0]} -> {x[1][1]} ({x[2]})"  , filter(lambda x: x[1] is not None, single_dependencies)))
single_correlated = list(map(lambda x: x[0], filter(lambda x: x[1] is None, single_dependencies)))
print("SINGLE SOFT DEPENDENCIES")
for i in single_softFD:
    print(i)
print("")
print("SINGLE CORRELATED")
for i in single_correlated:
    if not i[1] in transformed_columns and not i[0][0] in transformed_columns:
        print (f"{i[0][0]} <-> {i[1]}")
        continue
    if not i[1] in transformed_columns:
        print (f"{i[0][0]} -> {i[1]}")
        continue
    if not i[0][0] in transformed_columns:
        print (f"{i[1]} -> {i[0][0]}")


print("")
print("original length", len(q))
print("found SFD", len(single_softFD) )
print("found correlations", len(single_correlated))

SINGLE SOFT DEPENDENCIES
TaxiOut -> UniqueCarrier (1.0)
UniqueCarrier -> TaxiOut (1.0)

SINGLE CORRELATED
ActualElapsedTime <-> AirTime
ActualElapsedTime <-> ArrDelay
ActualElapsedTime <-> ArrTimestamp
ActualElapsedTime <-> CRSArrTimestamp
ActualElapsedTime <-> CRSDepTimeStamp
ActualElapsedTime <-> CRSElapsedTime
ActualElapsedTime <-> DepTimestamp
Dest -> ActualElapsedTime
ActualElapsedTime <-> Distance
ActualElapsedTime <-> NASAndWeatherDelay
Origin -> ActualElapsedTime
ActualElapsedTime <-> SecurityDelay
TailNum -> ActualElapsedTime
ActualElapsedTime <-> TaxiIn
ActualElapsedTime <-> TaxiOut
UniqueCarrier -> ActualElapsedTime
AirTime <-> ArrDelay
AirTime <-> CRSElapsedTime
Dest -> AirTime
AirTime <-> Distance
AirTime <-> LateAircraftAndCarrierDelay
AirTime <-> NASAndWeatherDelay
Origin -> AirTime
TailNum -> AirTime
AirTime <-> TaxiIn
AirTime <-> TaxiOut
UniqueCarrier -> AirTime
ArrDelay <-> ArrTimestamp
ArrDelay <-> CRSArrTimestamp
ArrDelay <-> CRSDepTimeStamp
ArrDelay <-> CRSElapsedT

Find Double dependecies 

In [10]:

single_softFDs = list(map(lambda x: (x[1][0], x[1][1]) , filter(lambda x: x[1] is not None, single_dependencies)))
q2 = list(filter(lambda x: not any(y[1] == x[-1] and set(y[0]).issubset(set(x[0])) for y in single_softFDs), q2))

# single_correlated is a list with tuples, the 0th index of the tuple is a list for the LHS the
# 1th index is a single column foro the RHS 
double_dependencies = find_correlected_dependencies_with_spark(q2)  

print(double_dependencies)

[((['ActualElapsedTime', 'AirTime'], 'ArrDelay'), None, 4.2936955026812357e-19), ((['ActualElapsedTime', 'AirTime'], 'CRSElapsedTime'), None, 0.0), ((['ActualElapsedTime', 'AirTime'], 'DepDelay'), None, 0.0004391437536158341), ((['ActualElapsedTime', 'AirTime'], 'Dest'), None, 0.00014236895724399748), ((['ActualElapsedTime', 'AirTime'], 'Distance'), None, 0.0), ((['ActualElapsedTime', 'AirTime'], 'LateAircraftAndCarrierDelay'), None, 0.008854890429680542), ((['ActualElapsedTime', 'AirTime'], 'NASAndWeatherDelay'), None, 4.2798043005055434e-64), ((['ActualElapsedTime', 'AirTime'], 'Origin'), None, 0.0011950423259930569), ((['ActualElapsedTime', 'AirTime'], 'SecurityDelay'), (['ActualElapsedTime', 'AirTime'], 'SecurityDelay'), 0.9987431264728986), ((['ActualElapsedTime', 'AirTime'], 'TailNum'), None, 0.0), ((['ActualElapsedTime', 'AirTime'], 'TaxiIn'), None, 2.395397145746435e-11), ((['ActualElapsedTime', 'AirTime'], 'TaxiOut'), None, 5.4564330142866916e-213), ((['ActualElapsedTime', 'Ai

In [11]:
# THIS IS SOME CODE TO DISPLAY THE RESULTS. NO INTERESTING LOGIC IS IN HERE
from pandas.api.types import is_numeric_dtype

double_softFD = list(map(lambda x: f"{x[1][0][0]} {x[1][0][1]}  -> {x[1][1]} ({x[2]})"  , filter(lambda x: x[1] is not None, double_dependencies)))
double_correlated = list(map(lambda x: x[0], filter(lambda x: x[1] is None, double_dependencies)))
print("DOUBLE SOFT DEPENDENCIES")
for i in double_softFD:
    print(i)
print("")
print("DOUBLE CORRELATED")
for i in double_correlated:
    if not i[1] in transformed_columns and not i[0][0] in transformed_columns:
        print (f"{i[0][0]} {i[0][1]} <-> {i[1]}")
        continue
    if not i[1] in transformed_columns:
        print (f"{i[0][0]} {i[0][1]} -> {i[1]}")
        continue
    if not i[0][0] in transformed_columns:
        print (f"{i[1]} -> {i[0][0]} {i[0][1]}")


print("")
print("original length", len(q2))
print("found SFD", len(double_softFD) )
print("found correlations", len(double_correlated))

DOUBLE SOFT DEPENDENCIES
ActualElapsedTime AirTime  -> SecurityDelay (0.9987431264728986)
ActualElapsedTime ArrDelay  -> SecurityDelay (0.9954522486104093)
ActualElapsedTime ArrTimestamp  -> CRSArrTimestamp (0.9997500937148569)
ActualElapsedTime ArrTimestamp  -> CRSDepTimeStamp (0.9995003123048095)
ActualElapsedTime ArrTimestamp  -> DepTimestamp (0.9995003123048095)
ActualElapsedTime ArrTimestamp  -> SecurityDelay (0.9991258741258742)
ActualElapsedTime ArrTimestamp  -> TaxiIn (0.9997500937148569)
ActualElapsedTime CRSArrTimestamp  -> ArrTimestamp (0.9997500937148569)
ActualElapsedTime CRSArrTimestamp  -> CRSDepTimeStamp (0.9997500937148569)
ActualElapsedTime CRSArrTimestamp  -> DepTimestamp (0.9997500937148569)
ActualElapsedTime CRSArrTimestamp  -> SecurityDelay (0.9991258741258742)
ActualElapsedTime CRSArrTimestamp  -> TaxiIn (0.9997500937148569)
ActualElapsedTime CRSDepTimeStamp  -> ArrTimestamp (0.9993753903810119)
ActualElapsedTime CRSDepTimeStamp  -> CRSArrTimestamp (0.99962514057

ArrTimestamp ArrDelay <-> ActualElapsedTime
ArrTimestamp ArrDelay <-> CRSElapsedTime
Dest -> ArrTimestamp ArrDelay
ArrTimestamp ArrDelay <-> Distance
Origin -> ArrTimestamp ArrDelay
TailNum -> ArrTimestamp ArrDelay
UniqueCarrier -> ArrTimestamp ArrDelay
Dest -> ArrTimestamp CRSArrTimestamp
Origin -> ArrTimestamp CRSArrTimestamp
TailNum -> ArrTimestamp CRSArrTimestamp
UniqueCarrier -> ArrTimestamp CRSArrTimestamp
ArrTimestamp CRSDepTimeStamp <-> ArrDelay
Dest -> ArrTimestamp CRSDepTimeStamp
Origin -> ArrTimestamp CRSDepTimeStamp
TailNum -> ArrTimestamp CRSDepTimeStamp
UniqueCarrier -> ArrTimestamp CRSDepTimeStamp
Dest -> ArrTimestamp CRSElapsedTime
Origin -> ArrTimestamp CRSElapsedTime
TailNum -> ArrTimestamp CRSElapsedTime
UniqueCarrier -> ArrTimestamp CRSElapsedTime
ArrTimestamp DepDelay <-> ArrDelay
Dest -> ArrTimestamp DepDelay
Origin -> ArrTimestamp DepDelay
TailNum -> ArrTimestamp DepDelay
UniqueCarrier -> ArrTimestamp DepDelay
ArrTimestamp DepTimestamp <-> ArrDelay
Dest -> ArrTim

NASAndWeatherDelay SecurityDelay <-> CRSDepTimeStamp
NASAndWeatherDelay SecurityDelay <-> DepDelay
NASAndWeatherDelay SecurityDelay <-> DepTimestamp
NASAndWeatherDelay SecurityDelay <-> LateAircraftAndCarrierDelay
TailNum -> NASAndWeatherDelay SecurityDelay
NASAndWeatherDelay SecurityDelay <-> TaxiOut
UniqueCarrier -> NASAndWeatherDelay SecurityDelay
NASAndWeatherDelay TailNum <-> ActualElapsedTime
NASAndWeatherDelay TailNum <-> AirTime
NASAndWeatherDelay TailNum <-> ArrDelay
NASAndWeatherDelay TailNum <-> ArrTimestamp
NASAndWeatherDelay TailNum <-> CRSArrTimestamp
NASAndWeatherDelay TailNum <-> CRSDepTimeStamp
NASAndWeatherDelay TailNum <-> CRSElapsedTime
NASAndWeatherDelay TailNum <-> DepDelay
NASAndWeatherDelay TailNum <-> DepTimestamp
Dest -> NASAndWeatherDelay TailNum
NASAndWeatherDelay TailNum <-> Distance
Origin -> NASAndWeatherDelay TailNum
NASAndWeatherDelay TailNum <-> TaxiOut
UniqueCarrier -> NASAndWeatherDelay TailNum
NASAndWeatherDelay TaxiIn <-> ActualElapsedTime
NASAndWe

In [12]:
#filter out subsets of already found fds.
print(len(q3))

double_softFDs = list(map(lambda x: (x[1][0], x[1][1]) , filter(lambda x: x[1] is not None, double_dependencies)))
q3 = list(filter(lambda x: not any(y[1] == x[-1] and set(y[0]).issubset(set(x[0])) for y in double_softFDs), q3))

print(len(q3))

# single_correlated is a list with tuples, the 0th index of the tuple is a list for the LHS the
# 1th index is a single column foro the RHS 
triple_dependencies = find_correlected_dependencies_with_spark(q3)  

print(triple_dependencies)

93024
55278


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
# THIS IS SOME CODE TO DISPLAY THE RESULTS. NO INTERESTING LOGIC IS IN HERE
from pandas.api.types import is_numeric_dtype

triple_softFD = list(map(lambda x: f"{x[1][0][0]} {x[1][0][1]} {x[1][0][2]} -> {x[1][1]} ({x[2]})"  , filter(lambda x: x[1] is not None, triple_dependencies)))
triple_correlated = list(map(lambda x: x[0], filter(lambda x: x[1] is None, triple_dependencies)))
print("Triple SOFT DEPENDENCIES")
for i in triple_softFD:
    print(i)
print("")
print("Triple CORRELATED")
for i in triple_correlated:
    if not i[1] in transformed_columns and not i[0][0] in transformed_columns:
        print (f"{i[0][0]} {i[0][1]} {i[0][2]} <-> {i[1]}")
        continue
    if not i[1] in transformed_columns:
        print (f"{i[0][0]} {i[0][1]} {i[0][2]} -> {i[1]}")
        continue
    if not i[0][0] in transformed_columns:
        print (f"{i[1]} -> {i[0][0]} {i[0][1]} {i[0][2]}")


print("")
print("original length", len(q3))
print("found SFD", len(triple_softFD) )
print("found correlations", len(triple_correlated))

Triple SOFT DEPENDENCIES
ActualElapsedTime AirTime ArrDelay -> NASAndWeatherDelay (0.9998009686193856)
ActualElapsedTime AirTime Dest -> NASAndWeatherDelay (0.9965184594182926)
ActualElapsedTime AirTime Dest -> TaxiOut (0.99565185883035)
ActualElapsedTime AirTime NASAndWeatherDelay -> TaxiOut (0.9941466420209488)
ActualElapsedTime AirTime Origin -> NASAndWeatherDelay (0.9967932905772077)
ActualElapsedTime AirTime Origin -> TaxiOut (0.9988465024305841)
ActualElapsedTime AirTime TaxiIn -> NASAndWeatherDelay (0.9985757240069632)
ActualElapsedTime AirTime TaxiIn -> TaxiOut (1.0)
ActualElapsedTime AirTime TaxiOut -> NASAndWeatherDelay (0.9902552204176334)
ActualElapsedTime AirTime UniqueCarrier -> DepDelay (0.9922638436482085)
ActualElapsedTime AirTime UniqueCarrier -> NASAndWeatherDelay (0.9982795346550877)
ActualElapsedTime AirTime UniqueCarrier -> TaxiOut (0.9995898277276456)
ActualElapsedTime ArrDelay AirTime -> NASAndWeatherDelay (0.9933655006031363)
ActualElapsedTime ArrDelay AirTime 

ActualElapsedTime SecurityDelay CRSArrTimestamp -> AirTime (0.9965899403239556)
ActualElapsedTime SecurityDelay CRSArrTimestamp -> ArrDelay (0.9985763097949886)
ActualElapsedTime SecurityDelay CRSArrTimestamp -> CRSElapsedTime (1.0)
ActualElapsedTime SecurityDelay CRSArrTimestamp -> DepDelay (0.9985763097949886)
ActualElapsedTime SecurityDelay CRSArrTimestamp -> Dest (0.9923599320882852)
ActualElapsedTime SecurityDelay CRSArrTimestamp -> Distance (0.9991452991452991)
ActualElapsedTime SecurityDelay CRSArrTimestamp -> NASAndWeatherDelay (0.9994300370475919)
ActualElapsedTime SecurityDelay CRSArrTimestamp -> TaxiOut (0.999714937286203)
ActualElapsedTime SecurityDelay CRSArrTimestamp -> UniqueCarrier (0.9951759364358683)
ActualElapsedTime SecurityDelay CRSDepTimeStamp -> AirTime (0.9965899403239556)
ActualElapsedTime SecurityDelay CRSDepTimeStamp -> ArrDelay (0.9985763097949886)
ActualElapsedTime SecurityDelay CRSDepTimeStamp -> CRSElapsedTime (1.0)
ActualElapsedTime SecurityDelay CRSDepT

AirTime SecurityDelay UniqueCarrier -> ActualElapsedTime (0.99644128113879)
AirTime SecurityDelay UniqueCarrier -> Distance (0.99644128113879)
AirTime SecurityDelay UniqueCarrier -> NASAndWeatherDelay (0.9946714031971581)
AirTime TailNum ActualElapsedTime -> CRSElapsedTime (0.9971563981042654)
AirTime TailNum ArrDelay -> ActualElapsedTime (0.9981060606060606)
AirTime TailNum ArrDelay -> CRSElapsedTime (0.9971617786187322)
AirTime TailNum ArrDelay -> Distance (0.996219281663516)
AirTime TailNum Origin -> ActualElapsedTime (0.998193315266486)
AirTime TailNum Origin -> CRSElapsedTime (0.9972924187725631)
AirTime TailNum Origin -> Distance (0.9963931469792606)
AirTime TailNum SecurityDelay -> ActualElapsedTime (0.9981006647673314)
AirTime TailNum SecurityDelay -> Distance (0.9962085308056872)
AirTime TailNum TaxiIn -> ActualElapsedTime (0.9980988593155894)
AirTime TailNum TaxiIn -> CRSElapsedTime (0.9971509971509972)
AirTime TailNum TaxiIn -> Distance (0.9962049335863378)
AirTime TailNum T

CRSElapsedTime ActualElapsedTime LateAircraftAndCarrierDelay -> AirTime (1.0)
CRSElapsedTime ActualElapsedTime LateAircraftAndCarrierDelay -> ArrDelay (1.0)
CRSElapsedTime ActualElapsedTime LateAircraftAndCarrierDelay -> ArrTimestamp (1.0)
CRSElapsedTime ActualElapsedTime LateAircraftAndCarrierDelay -> CRSArrTimestamp (1.0)
CRSElapsedTime ActualElapsedTime LateAircraftAndCarrierDelay -> CRSDepTimeStamp (1.0)
CRSElapsedTime ActualElapsedTime LateAircraftAndCarrierDelay -> DepDelay (1.0)
CRSElapsedTime ActualElapsedTime LateAircraftAndCarrierDelay -> DepTimestamp (1.0)
CRSElapsedTime ActualElapsedTime LateAircraftAndCarrierDelay -> Dest (1.0)
CRSElapsedTime ActualElapsedTime LateAircraftAndCarrierDelay -> Distance (1.0)
CRSElapsedTime ActualElapsedTime LateAircraftAndCarrierDelay -> NASAndWeatherDelay (1.0)
CRSElapsedTime ActualElapsedTime LateAircraftAndCarrierDelay -> Origin (1.0)
CRSElapsedTime ActualElapsedTime LateAircraftAndCarrierDelay -> TailNum (1.0)
CRSElapsedTime ActualElapsed

CRSElapsedTime CRSArrTimestamp LateAircraftAndCarrierDelay -> TailNum (1.0)
CRSElapsedTime CRSArrTimestamp LateAircraftAndCarrierDelay -> TaxiOut (1.0)
CRSElapsedTime CRSArrTimestamp LateAircraftAndCarrierDelay -> UniqueCarrier (1.0)
CRSElapsedTime CRSArrTimestamp NASAndWeatherDelay -> AirTime (1.0)
CRSElapsedTime CRSArrTimestamp NASAndWeatherDelay -> ArrDelay (1.0)
CRSElapsedTime CRSArrTimestamp NASAndWeatherDelay -> DepDelay (1.0)
CRSElapsedTime CRSArrTimestamp NASAndWeatherDelay -> Dest (1.0)
CRSElapsedTime CRSArrTimestamp NASAndWeatherDelay -> Distance (1.0)
CRSElapsedTime CRSArrTimestamp NASAndWeatherDelay -> LateAircraftAndCarrierDelay (1.0)
CRSElapsedTime CRSArrTimestamp NASAndWeatherDelay -> Origin (1.0)
CRSElapsedTime CRSArrTimestamp NASAndWeatherDelay -> TailNum (1.0)
CRSElapsedTime CRSArrTimestamp NASAndWeatherDelay -> TaxiOut (1.0)
CRSElapsedTime CRSArrTimestamp NASAndWeatherDelay -> UniqueCarrier (1.0)
CRSElapsedTime CRSArrTimestamp Origin -> DepDelay (1.0)
CRSElapsedTime 

...

SecurityDelay Dest CRSArrTimestamp <-> ArrDelay
SecurityDelay Dest CRSArrTimestamp <-> CRSElapsedTime
SecurityDelay Dest CRSArrTimestamp <-> DepDelay
SecurityDelay Dest CRSArrTimestamp <-> Distance
SecurityDelay Dest CRSArrTimestamp <-> LateAircraftAndCarrierDelay
SecurityDelay Dest CRSArrTimestamp <-> NASAndWeatherDelay
SecurityDelay Dest CRSArrTimestamp <-> TaxiOut
UniqueCarrier -> SecurityDelay Dest CRSArrTimestamp
SecurityDelay Dest CRSDepTimeStamp <-> ActualElapsedTime
SecurityDelay Dest CRSDepTimeStamp <-> AirTime
SecurityDelay Dest CRSDepTimeStamp <-> CRSElapsedTime
SecurityDelay Dest CRSDepTimeStamp <-> Distance
SecurityDelay Dest CRSDepTimeStamp <-> TaxiOut
UniqueCarrier -> SecurityDelay Dest CRSDepTimeStamp
SecurityDelay Dest CRSElapsedTime <-> AirTime
SecurityDelay Dest CRSElapsedTime <-> ArrDelay
SecurityDelay Dest CRSElapsedTime <-> DepDelay
SecurityDelay Dest CRSElapsedTime <-> LateAircraftAndCarrierDelay
SecurityDelay Dest CRSElapsedTime <-> NASAndWeatherDelay
SecurityDe

SecurityDelay Origin UniqueCarrier <-> DepDelay
Dest -> SecurityDelay Origin UniqueCarrier
SecurityDelay Origin UniqueCarrier <-> Distance
SecurityDelay Origin UniqueCarrier <-> LateAircraftAndCarrierDelay
SecurityDelay Origin UniqueCarrier <-> NASAndWeatherDelay
SecurityDelay Origin UniqueCarrier <-> TaxiIn
SecurityDelay Origin UniqueCarrier <-> TaxiOut
SecurityDelay TailNum ActualElapsedTime <-> CRSArrTimestamp
SecurityDelay TailNum ActualElapsedTime <-> CRSDepTimeStamp
SecurityDelay TailNum ActualElapsedTime <-> DepTimestamp
Dest -> SecurityDelay TailNum ActualElapsedTime
SecurityDelay TailNum ActualElapsedTime <-> NASAndWeatherDelay
Origin -> SecurityDelay TailNum ActualElapsedTime
UniqueCarrier -> SecurityDelay TailNum ActualElapsedTime
SecurityDelay TailNum AirTime <-> ActualElapsedTime
SecurityDelay TailNum AirTime <-> ArrTimestamp
SecurityDelay TailNum AirTime <-> Distance
UniqueCarrier -> SecurityDelay TailNum AirTime
Origin -> SecurityDelay TailNum ArrDelay
SecurityDelay Tail

TailNum TaxiIn UniqueCarrier -> Distance
TailNum TaxiIn UniqueCarrier -> LateAircraftAndCarrierDelay
TailNum TaxiIn UniqueCarrier -> NASAndWeatherDelay
TailNum TaxiIn UniqueCarrier -> TaxiOut
TailNum TaxiOut ActualElapsedTime -> ArrTimestamp
TailNum TaxiOut ActualElapsedTime -> CRSArrTimestamp
TailNum TaxiOut ActualElapsedTime -> CRSDepTimeStamp
TailNum TaxiOut ActualElapsedTime -> DepTimestamp
TailNum TaxiOut AirTime -> ActualElapsedTime
TailNum TaxiOut AirTime -> ArrTimestamp
TailNum TaxiOut AirTime -> CRSElapsedTime
TailNum TaxiOut TaxiIn -> Distance
TailNum UniqueCarrier ActualElapsedTime -> CRSElapsedTime
TailNum UniqueCarrier AirTime -> ActualElapsedTime
TailNum UniqueCarrier AirTime -> CRSElapsedTime
TailNum UniqueCarrier AirTime -> Distance
TailNum UniqueCarrier Origin -> CRSElapsedTime
TailNum UniqueCarrier Origin -> Distance
TailNum UniqueCarrier SecurityDelay -> ActualElapsedTime
TailNum UniqueCarrier SecurityDelay -> AirTime
TailNum UniqueCarrier SecurityDelay -> ArrTimesta

TaxiIn Distance NASAndWeatherDelay <-> DepTimestamp
TaxiIn Distance Origin <-> ArrTimestamp
TaxiIn Distance Origin <-> CRSArrTimestamp
TaxiIn Distance Origin <-> CRSDepTimeStamp
TaxiIn Distance Origin <-> DepTimestamp
TaxiIn Distance SecurityDelay <-> CRSDepTimeStamp
TaxiIn Distance SecurityDelay <-> DepTimestamp
TaxiIn Distance TaxiOut <-> CRSDepTimeStamp
TaxiIn Distance TaxiOut <-> DepTimestamp
TaxiIn Distance UniqueCarrier <-> CRSDepTimeStamp
TaxiIn Distance UniqueCarrier <-> DepTimestamp
TaxiIn LateAircraftAndCarrierDelay ActualElapsedTime <-> CRSDepTimeStamp
TaxiIn LateAircraftAndCarrierDelay ActualElapsedTime <-> DepTimestamp
TaxiIn LateAircraftAndCarrierDelay AirTime <-> CRSDepTimeStamp
TaxiIn LateAircraftAndCarrierDelay AirTime <-> DepTimestamp
TaxiIn LateAircraftAndCarrierDelay ArrDelay <-> CRSDepTimeStamp
TaxiIn LateAircraftAndCarrierDelay ArrDelay <-> DepTimestamp
TailNum -> TaxiIn LateAircraftAndCarrierDelay ArrTimestamp
TailNum -> TaxiIn LateAircraftAndCarrierDelay CRSArrT

TaxiOut Distance NASAndWeatherDelay <-> CRSArrTimestamp
TaxiOut Distance NASAndWeatherDelay <-> CRSDepTimeStamp
TaxiOut Distance NASAndWeatherDelay <-> DepTimestamp
TailNum -> TaxiOut Distance NASAndWeatherDelay
UniqueCarrier -> TaxiOut Distance NASAndWeatherDelay
TaxiOut Distance Origin <-> AirTime
TailNum -> TaxiOut Distance Origin
UniqueCarrier -> TaxiOut Distance Origin
TaxiOut Distance SecurityDelay <-> AirTime
TaxiOut Distance SecurityDelay <-> ArrTimestamp
TaxiOut Distance SecurityDelay <-> CRSArrTimestamp
TaxiOut Distance SecurityDelay <-> CRSDepTimeStamp
TaxiOut Distance SecurityDelay <-> DepTimestamp
TailNum -> TaxiOut Distance SecurityDelay
UniqueCarrier -> TaxiOut Distance SecurityDelay
TaxiOut Distance TaxiIn <-> AirTime
TaxiOut Distance TaxiIn <-> ArrTimestamp
TaxiOut Distance TaxiIn <-> CRSArrTimestamp
TaxiOut Distance TaxiIn <-> CRSDepTimeStamp
TaxiOut Distance TaxiIn <-> DepTimestamp
TailNum -> TaxiOut Distance TaxiIn
UniqueCarrier -> TaxiOut Distance TaxiIn
TaxiOut Di

This is the interface to the code of Twan, Teun and Ruben

In [14]:
to_check_sfd = list(map(lambda x: [x[1][0][0], x[1][1]] , filter(lambda x: x[1] is not None, single_dependencies)))
to_check_sfd += list(map(lambda x: [x[1][0][0],x[1][0][1], x[1][1]] , filter(lambda x: x[1] is not None, double_dependencies)))
to_check_sfd += list(map(lambda x: [x[1][0][0],x[1][0][1],x[1][0][2], x[1][1]] , filter(lambda x: x[1] is not None, triple_dependencies)))

to_check_delta = []
for i in list( filter(lambda x: x[1] is None, single_dependencies)):
    if not i[1] in transformed_columns and not i[0][0] in transformed_columns:
        to_check_delta.append([i[0][0], i[1]])
        to_check_delta.append([i[1], i[0][0]])
        continue
    if not i[1] in transformed_columns:
        to_check_delta.append([i[0][0], i[1]])
        continue
    if not i[0][0] in transformed_columns:
        to_check_delta.append([i[1], i[0][0]])

to_check_delta += list(map(lambda x: [x[0][0][0], x[0][0][1], x[0][1]], filter(lambda x: x[1] is None and not i[0][1] in transformed_columns, double_dependencies)))
to_check_delta += list(map(lambda x: [x[0][0][0], x[0][0][1], x[0][0][2], x[0][1]], filter(lambda x: x[1] is None and not i[0][1] in transformed_columns, triple_dependencies)))